In [1]:
import numpy as np
import scipy as sp
from scipy import optimize,special,integrate,stats
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

import copy

import math 

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

# $x/|x|$

In [3]:
Λ = np.hstack([-model_problem_spectrum(100,.9,1e2)[::-1],model_problem_spectrum(300,.8,1e3)])
A = np.diag(Λ)

n = len(A)
b = np.ones(n,dtype=np.double)
b /= np.linalg.norm(b)

f = lambda x: x/abs(x)#(A*x**2+B*x+C)/(a*x**2+b*x+c)
fAb = f(Λ)*b

In [4]:
K = 100
Q,(a_,b_) = lanczos_reorth(A,b,K,reorth=K)

In [5]:
error_opt = np.full(K,np.nan)
error_FA = np.full(K,np.nan)
error_OR = np.full(K,np.nan)
error_HA = np.full(K,np.nan)


for i in range(1,K):
    
    opt = Q[:,:i]@np.linalg.solve(Q[:,:i].T@np.diag(Λ**2)@Q[:,:i],Q[:,:i].T@(Λ**2*fAb))
    error_opt[i] = np.linalg.norm(Λ*(fAb - opt))
    
    T = np.diag(a_[:i]) + np.diag(b_[:i-1],1) + np.diag(b_[:i-1],-1)
    theta,S = sp.linalg.eigh(T)
    
    lank_FA = Q[:,:i]@(S@(f(theta)*S[0]))
    error_FA[i] = np.linalg.norm(Λ*(fAb - lank_FA))
   
    TT_ = T@T
    TT_[-1,-1] += b_[i-1]**2
    theta,S = sp.linalg.eigh(TT_)
    
    lank_OR = Q[:,:i]@(S@((1/np.sqrt(theta))*(S.T@T[0])))
    error_OR[i] = np.linalg.norm(Λ*(fAb - lank_OR))
   
    T_ = np.copy(T)

    ek1 = np.zeros(i)
    ek1[-1] = 1
    e0 = np.zeros(i)
    e0[0] = 1
    
    z = b_[i-1]**2*np.linalg.solve(T,ek1)
    T_[:,-1] += z
    theta,S = sp.linalg.eig(T_)

    hark = Q[:,:i]@(S@(f(np.real(theta))*np.linalg.solve(S,e0)))
    error_HA[i] = np.linalg.norm(Λ*(fAb - hark))
    

In [6]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))
ax = fig.add_axes([left, bottom, width, height])

skip = 2
ks = np.arange(K)[::skip]

ax.plot(ks,(error_opt/error_opt)[::skip],\
        color=colors[4],marker=markers[4],ms=m_sizes[4],lw=.2)
#ax.plot(ks,(a_posteriori_bound_wedge_approx_spectrum*err_CG)[::skip],\
#        color=colors[2],marker=markers[2],ms=m_sizes[2],lw=.2)

ax.plot(ks,(error_HA/error_opt)[::skip],\
        color=colors[3],marker=markers[3],ms=m_sizes[3],lw=.2)
ax.plot(ks,(error_FA/error_opt)[::skip],\
        color=colors[2],marker=markers[2],ms=m_sizes[2],lw=.2)

ax.plot(ks,(error_OR/error_opt)[::skip],\
        color=colors[1],marker=markers[1],ms=m_sizes[1],lw=.2)


#ax.set_yscale('log')

plt.grid(True,linestyle=':',linewidth=.5)

plt.ylim(.9,1.5)

#plt.title(f'$\mathbf{{A}}$-norm error bounds: $f(x) = \sqrt{{x}}$, circle contour')
plt.xlabel('number of matvecs: $k$')

plt.savefig('imgs/ch6_sign.pdf')
#plt.close()